In [1]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict
from datetime import timedelta
import math
import copy

# Set random seed for reproducibility
random.seed(20)

# --- Data Loading ---
file_path = "generic_input_case.xlsx" # Path to the input data

# Load data from Excel sheets into DataFrames
df_horizon = pd.read_excel(file_path, sheet_name="HORIZONTE") # Planning horizon
df_up_database = pd.read_excel(file_path, sheet_name="BD_UP") # Production Unit details
df_fleet = pd.read_excel(file_path, sheet_name="FROTA")       # Transporter fleet info
df_crane = pd.read_excel(file_path, sheet_name="GRUA")       # Crane capacity/rules
df_factory = pd.read_excel(file_path, sheet_name="FABRICA")   # Factory constraints
df_route = pd.read_excel(file_path, sheet_name="ROTA")       # Route details (UP -> Transporter)





c:\Users\wilma\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [2]:

# --- Constraint Validation Functions ---

def validate_daily_demand(schedule_df, df_factory, flag):
    """Checks if daily delivered volume meets factory demand constraints."""
    daily_volume = schedule_df.groupby('DIA')['VOLUME'].sum()
    merged = pd.merge(daily_volume.reset_index(), df_factory[['DIA', 'DEMANDA_MIN', 'DEMANDA_MAX']], on='DIA')

    # Check for violations
    violations = merged[
        (merged['VOLUME'] < merged['DEMANDA_MIN']) |
        (merged['VOLUME'] > merged['DEMANDA_MAX'])
    ]

    if not violations.empty:
        if flag == 1: # Optionally print details
            print("Violation - Daily Demand:")
            print(violations)
        return False
    return True

def validate_daily_rsp(schedule_df, df_factory, flag):
    """Checks if the daily weighted average RSP is within factory limits."""
    if schedule_df.empty or 'VOLUME' not in schedule_df.columns or 'RSP' not in schedule_df.columns:
        return True # Pass if no relevant data

    # Calculate weighted average RSP per day
    schedule_df['RSP_WEIGHTED_SUM'] = schedule_df['RSP'] * schedule_df['VOLUME']
    daily_rsp_sum = schedule_df.groupby('DIA')['RSP_WEIGHTED_SUM'].sum()
    daily_volume_sum = schedule_df.groupby('DIA')['VOLUME'].sum()
    daily_avg_rsp = (daily_rsp_sum / daily_volume_sum).fillna(0).reset_index()
    daily_avg_rsp.rename(columns={0: 'RSP_AVG'}, inplace=True)

    # Compare against limits
    merged = pd.merge(daily_avg_rsp, df_factory[['DIA', 'RSP_MIN', 'RSP_MAX']], on='DIA')
    violations = merged[
        (round(merged['RSP_AVG'], 2) < round(merged['RSP_MIN'], 2)) |
        (round(merged['RSP_AVG'], 2) > round(merged['RSP_MAX'], 2))
    ]

    if not violations.empty:
        if flag == 1: # Optionally print details
            print("Violation - Daily RSP:")
            print(violations)
        return False
    return True

def validate_transporter_fleet_size(schedule_df, df_fleet):
    """Checks if daily vehicle usage per transporter is within their fleet limits."""
    daily_fleet = schedule_df.groupby(['DIA', 'TRANSPORTADOR'])['QTD_VEICULOS'].sum().reset_index()
    merged = pd.merge(daily_fleet, df_fleet[['TRANSPORTADOR', 'FROTA_MIN', 'FROTA_MAX']], on='TRANSPORTADOR')

    # Check for violations
    violations = merged[
        (merged['QTD_VEICULOS'] < merged['FROTA_MIN']) |
        (merged['QTD_VEICULOS'] > merged['FROTA_MAX'])
    ]

    if not violations.empty:
        print("Violation - Transporter Fleet Size:")
        print(violations)
        return False
    return True

def validate_crane_capacity(schedule_df, df_crane):
    """Checks if a transporter serves more UPs simultaneously than their crane capacity."""
    daily_up_count = schedule_df.groupby(['DIA', 'TRANSPORTADOR'])['UP'].nunique().reset_index()
    daily_up_count.rename(columns={'UP': 'UP_COUNT'}, inplace=True)
    merged = pd.merge(daily_up_count, df_crane[['TRANSPORTADOR', 'QTD_GRUAS']], on='TRANSPORTADOR')

    # Check for violations
    violations = merged[merged['UP_COUNT'] > merged['QTD_GRUAS']]

    if not violations.empty:
        print("Violation - Crane Capacity:")
        print(violations)
        return False
    return True

def validate_fazenda_exclusivity(schedule_df):
    """Checks if a transporter operates in more than one Fazenda per day."""
    daily_fazenda_count = schedule_df.groupby(['DIA', 'TRANSPORTADOR'])['FAZENDA'].nunique().reset_index()
    daily_fazenda_count.rename(columns={'FAZENDA': 'FAZENDA_COUNT'}, inplace=True)

    # Check for violations
    violations = daily_fazenda_count[daily_fazenda_count['FAZENDA_COUNT'] > 1]

    if not violations.empty:
        print("Violation - Fazenda Exclusivity:")
        print(violations)
        return False
    return True

def validate_min_vehicles_per_up(schedule_df, df_crane):
    """Checks if the minimum vehicle percentage per UP assignment is met."""
    total_daily_fleet = schedule_df.groupby(['DIA', 'TRANSPORTADOR'])['QTD_VEICULOS'].sum().reset_index()
    total_daily_fleet.rename(columns={'QTD_VEICULOS': 'TOTAL_VEHICULOS_DIA'}, inplace=True)

    merged = pd.merge(schedule_df, total_daily_fleet, on=['DIA', 'TRANSPORTADOR'])
    merged = pd.merge(merged, df_crane[['TRANSPORTADOR', 'PORCENTAGEM_VEICULOS_MIN']], on='TRANSPORTADOR')

    # Calculate actual percentage, handling potential division by zero
    merged['PORCENTAJE_REAL'] = np.where(merged['TOTAL_VEHICULOS_DIA'] > 0,
                                         merged['QTD_VEICULOS'] / merged['TOTAL_VEHICULOS_DIA'],
                                         0)

    # Check for violations (using tolerance)
    tolerance = 1e-6
    violations = merged[merged['PORCENTAJE_REAL'] < merged['PORCENTAGEM_VEICULOS_MIN'] - tolerance]

    if not violations.empty:
        print("Violation - Minimum Vehicles per UP:")
        print(violations[['DIA', 'TRANSPORTADOR', 'UP', 'QTD_VEICULOS', 'TOTAL_VEHICULOS_DIA', 'PORCENTAGEM_VEICULOS_MIN', 'PORCENTAJE_REAL']])
        return False
    return True


def check_continuity(days_list):
    """Helper to check work continuity and count work blocks for a list of days."""
    if not isinstance(days_list, list) or not days_list:
         return 0, True # No days worked -> 0 blocks, continuous

    # Filter NaNs, get unique days, sort
    sorted_days = sorted(list(set(filter(lambda x: x is not None and not pd.isna(x), days_list))))
    if not sorted_days:
      return 0, True # No valid days -> 0 blocks, continuous

    blocks = 1 # Start assuming one block
    for i in range(len(sorted_days) - 1):
        if sorted_days[i+1] != sorted_days[i] + 1: # Check for gap
            blocks += 1
    is_continuous = (blocks <= 1)
    return blocks, is_continuous

def validate_completion_rules(schedule_df, df_up_database):
    """Validates UP completion rules based on size (<7000m³ vs >=7000m³)."""
    all_valid = True
    total_volume_transported = schedule_df.groupby('UP')['VOLUME'].sum()

    for up_id, group in schedule_df.groupby('UP'):
        try:
            up_info = df_up_database[df_up_database['UP'] == up_id].iloc[0]
            total_volume_up = up_info['VOLUME']
            volume_transported_up = total_volume_transported.get(up_id, 0)
            days_worked = group['DIA'].tolist()
            num_blocks, _ = check_continuity(days_worked) # Use check_continuity helper

            is_small_up = total_volume_up < 7000

            # Rule: Small UP (<7000 m³) - must be completed if started, in 1 block
            if is_small_up:
                if 0 < volume_transported_up < total_volume_up:
                     print(f"Violation UP < 7000: UP {up_id} incomplete. "
                           f"Transported: {volume_transported_up:.2f}, Total: {total_volume_up:.2f}")
                     all_valid = False
                if num_blocks > 1:
                     print(f"Violation UP < 7000: UP {up_id} has {num_blocks} blocks (max 1). "
                           f"Days: {sorted(list(set(days_worked)))}")
                     all_valid = False

            # Rule: Large UP (>=7000 m³) - max 2 blocks allowed
            else:
                if num_blocks > 2:
                     print(f"Violation UP >= 7000: UP {up_id} has {num_blocks} blocks (max 2). "
                           f"Days: {sorted(list(set(days_worked)))}")
                     all_valid = False
        except IndexError:
            print(f"Warning: Missing info for UP {up_id}. Skipping completion check.")
            continue
        except Exception as e:
            print(f"Error validating completion for UP {up_id}: {e}")
            all_valid = False

    # Note: Fazenda completion rules are more complex and not fully validated here.
    return all_valid

# --- Main Validation Wrappers ---

def validate_essential_constraints(schedule_df, df_factory, df_fleet, df_crane, df_up_database, flag):
    """Runs essential validations (demand, RSP, fleet, crane, exclusivity)."""
    is_feasible = True
    temp_schedule = schedule_df.copy() # Work on a copy to avoid modifying original

    if 'RSP_WEIGHTED_SUM' in temp_schedule.columns:
        temp_schedule.drop(columns=['RSP_WEIGHTED_SUM'], inplace=True, errors='ignore')

    if not validate_daily_demand(temp_schedule, df_factory, flag): is_feasible = False
    if not validate_daily_rsp(temp_schedule, df_factory, flag): is_feasible = False # Modifies temp_schedule
    if not validate_transporter_fleet_size(temp_schedule, df_fleet): is_feasible = False
    if not validate_crane_capacity(temp_schedule, df_crane): is_feasible = False
    if not validate_fazenda_exclusivity(temp_schedule): is_feasible = False

    return is_feasible

# Full validation including completion rules (can be slower)
# def validate_all_constraints(schedule_df, df_factory, df_fleet, df_crane, df_up_database, flag):
#     """Runs all validations including completion rules."""
#     print("--- Starting Full Constraint Validation ---")
#     is_feasible = validate_essential_constraints(schedule_df, df_factory, df_fleet, df_crane, df_up_database, flag)
#
#     if not validate_completion_rules(schedule_df, df_up_database): is_feasible = False
#     # Add validate_min_vehicles_per_up if needed, ensuring schedule_df is comprehensive
#     # try:
#     #     if not validate_min_vehicles_per_up(schedule_df, df_crane): is_feasible = False
#     # except Exception as e: is_feasible = False; print(f"Min vehicles check failed: {e}")
#
#     print("--- Validation Finished ---")
#     status = "MEETS" if is_feasible else "DOES NOT MEET"
#     print(f">>> Solution {status} all verified constraints.")
#     return is_feasible

In [3]:

# --- Auxiliary & Objective Functions ---

def calculate_daily_transport_volume(vehicles, load_capacity, cycle_time):
    """Calculates potential daily volume for a given assignment."""
    if vehicles <= 0 or load_capacity <= 0 or cycle_time <= 0:
        return 0
    return vehicles * load_capacity * cycle_time

def calculate_objective(schedule_df):
    """Calculates the average daily DB variation (objective function)."""
    if schedule_df is None or schedule_df.empty or 'DIA' not in schedule_df.columns or 'DB' not in schedule_df.columns:
        return float('inf')

    total_variation = 0
    # Use unique days present in the schedule for averaging
    days_in_schedule = schedule_df['DIA'].unique()
    num_days_total = len(days_in_schedule) if len(days_in_schedule) > 0 else 1 # Avoid division by zero

    # Calculate variation only for days with data
    daily_stats = schedule_df.groupby('DIA')['DB'].agg(['min', 'max'])
    daily_stats.dropna(inplace=True) # Ignore days with NaN DBs
    total_variation = (daily_stats['max'] - daily_stats['min']).sum()

    # Average over the number of unique days in the schedule
    average_variation = total_variation / num_days_total
    return average_variation

In [4]:

# --- Constructive Heuristic ---

def constructive_heuristic(df_horizon, df_up_db_input, df_fleet, df_crane, df_factory, df_route, flag):
    """Builds an initial schedule day-by-day using greedy choices."""
    schedule_entries = [] # Use a more descriptive name

    # Initialize UP status tracker
    up_status = df_up_db_input.set_index('UP').copy()
    up_status['VOLUME_RESTANTE'] = up_status['VOLUME']
    up_status['DIAS_TRABAJADOS'] = [[] for _ in range(len(up_status))]
    up_status['BLOQUES_TRABAJO'] = 0
    up_status['ESTADO'] = 'PENDIENTE' # PENDING, ACTIVA, COMPLETADA, PAUSADA

    # Track transporter state
    transporter_last_fazenda = {t: None for t in df_fleet['TRANSPORTADOR']}
    transporters = list(df_fleet['TRANSPORTADOR'].unique())
    random.shuffle(transporters) # Randomize transporter order

    up_status = up_status.sample(frac=1) # Randomize UP order

    # --- Daily Loop ---
    for _, day_info in df_horizon.iterrows():
        current_day = day_info['DIA']
        current_month = day_info['MES']

        # Prioritize UPs: active/paused first, then pending
        ups_active_paused = up_status[up_status['ESTADO'].isin(['ACTIVA', 'PAUSADA'])].index.tolist()
        ups_pending = up_status[up_status['ESTADO'] == 'PENDIENTE'].index.tolist()
        prioritized_ups = ups_active_paused + ups_pending

        # --- Transporter Loop ---
        for transporter in transporters:
            try:
                t_info = df_fleet[df_fleet['TRANSPORTADOR'] == transporter].iloc[0]
                c_info = df_crane[df_crane['TRANSPORTADOR'] == transporter].iloc[0]
                fleet_min, fleet_max = t_info['FROTA_MIN'], t_info['FROTA_MAX']
                num_cranes = c_info['QTD_GRUAS']
                min_veh_perc = c_info["PORCENTAGEM_VEICULOS_MIN"]
            except IndexError:
                 continue

            # --- Determine Target Fazenda (prioritize continuity) ---
            target_fazenda = None
            prev_fazenda = transporter_last_fazenda[transporter]
            if prev_fazenda:
                 ups_in_prev = up_status[up_status['FAZENDA'] == prev_fazenda]
                 if any(ups_in_prev['ESTADO'].isin(['ACTIVA', 'PAUSADA'])):
                     target_fazenda = prev_fazenda

            # If no continuity needed, find a new Fazenda
            if not target_fazenda:
                 possible_fazendas = up_status[up_status.index.isin(prioritized_ups)]['FAZENDA'].unique()
                 random.shuffle(possible_fazendas)
                 for f in possible_fazendas:
                     ups_in_f = up_status[(up_status['FAZENDA'] == f) & up_status.index.isin(prioritized_ups)].index
                     # Check if transporter has a route to any prioritized UP in this fazenda
                     if any(not df_route[(df_route['ORIGEM'] == up) & (df_route['TRANSPORTADOR'] == transporter)].empty for up in ups_in_f):
                          target_fazenda = f
                          break

            if not target_fazenda:
                transporter_last_fazenda[transporter] = None
                continue # No suitable Fazenda found for this transporter today

            transporter_last_fazenda[transporter] = target_fazenda # Update last worked

            # --- Select UPs within the Target Fazenda (respecting crane limit) ---
            selected_ups_today = []
            candidate_ups = [up for up in prioritized_ups if up_status.loc[up, 'FAZENDA'] == target_fazenda]

            for up_id in candidate_ups:
                if len(selected_ups_today) >= num_cranes: break # Crane limit

                # Check route exists
                if df_route[(df_route['ORIGEM'] == up_id) & (df_route['TRANSPORTADOR'] == transporter)].empty:
                    continue

                # Check completion rules feasibility (simplified check)
                try:
                    up_data = up_status.loc[up_id]
                    if up_data['VOLUME_RESTANTE'] <= 1e-6: continue # Already done

                    can_process = False
                    state = up_data['ESTADO']
                    blocks = up_data['BLOQUES_TRABAJO']
                    is_small = up_data['VOLUME'] < 7000

                    if state == 'PENDIENTE':
                        can_process = True if is_small else (blocks < 2)
                    elif state == 'ACTIVA':
                        can_process = True
                    elif state == 'PAUSADA':
                        can_process = (blocks < 2)

                    if can_process:
                        selected_ups_today.append(up_id)
                except KeyError: continue # Skip if UP data missing

            if not selected_ups_today: continue # No UPs selected

            # --- Assign Vehicles ---
            num_ups = len(selected_ups_today)
            vehicles_per_up = {}
            # Simple heuristic: assign max fleet, distribute somewhat evenly respecting min %
            total_vehicles = fleet_max
            min_req_each = math.ceil(min_veh_perc * total_vehicles)
            base_assign = max(1, min_req_each)
            remaining_veh = total_vehicles
            temp_assignment = {}

            # Assign base minimum
            for up_id in selected_ups_today:
                 assign_count = min(base_assign, remaining_veh)
                 if assign_count > 0:
                     temp_assignment[up_id] = assign_count
                     remaining_veh -= assign_count
                 else: temp_assignment[up_id] = 0

            # Distribute remainder
            up_order = list(selected_ups_today)
            random.shuffle(up_order)
            idx = 0
            while remaining_veh > 0 and num_ups > 0:
                 up_to_inc = up_order[idx % num_ups]
                 temp_assignment[up_to_inc] += 1
                 remaining_veh -= 1
                 idx += 1

            # Final check: ensure assigned vehicles can transport volume & respect fleet total
            final_veh_assigned_today = 0
            for up_id in selected_ups_today:
                assigned_veh = temp_assignment.get(up_id, 0)
                if assigned_veh <= 0: continue
                try:
                    route_info = df_route[(df_route['ORIGEM'] == up_id) & (df_route['TRANSPORTADOR'] == transporter)].iloc[0]
                    vol = calculate_daily_transport_volume(assigned_veh, route_info['CAIXA_CARGA'], route_info['TEMPO_CICLO'])
                    if vol > 1e-6:
                        vehicles_per_up[up_id] = assigned_veh
                        final_veh_assigned_today += assigned_veh
                except IndexError: continue # Skip if no route info

            # Recalculate selected UPs based on valid assignments
            selected_ups_today = list(vehicles_per_up.keys())

            # --- Calculate Transported Volume & Update State ---
            for up_id in selected_ups_today:
                assigned_vehicles = vehicles_per_up[up_id]
                try:
                    up_data = up_status.loc[up_id]
                    route_info = df_route[(df_route['ORIGEM'] == up_id) & (df_route['TRANSPORTADOR'] == transporter)].iloc[0]
                    max_vol_today = calculate_daily_transport_volume(assigned_vehicles, route_info['CAIXA_CARGA'], route_info['TEMPO_CICLO'])
                    transported_vol = min(max_vol_today, up_data['VOLUME_RESTANTE'])

                    if transported_vol > 1e-6:
                        # Update UP state
                        up_status.loc[up_id, 'VOLUME_RESTANTE'] -= transported_vol
                        if up_status.loc[up_id, 'VOLUME_RESTANTE'] < 0: up_status.loc[up_id, 'VOLUME_RESTANTE'] = 0

                        # Update work days and blocks safely
                        days_worked = up_status.at[up_id, 'DIAS_TRABAJADOS']
                        if not isinstance(days_worked, list): days_worked = []
                        last_day = days_worked[-1] if days_worked else None
                        is_new_block = (last_day is None) or (current_day != last_day + 1)

                        if is_new_block and last_day is not None: # Increment block count if gap
                            if pd.isna(up_status.loc[up_id, 'BLOQUES_TRABAJO']): up_status.loc[up_id, 'BLOQUES_TRABAJO'] = 0
                            up_status.loc[up_id, 'BLOQUES_TRABAJO'] += 1

                        if current_day not in days_worked: # Add current day
                            days_worked.append(current_day)
                            up_status.at[up_id, 'DIAS_TRABAJADOS'] = days_worked # Assign back

                        # Set final state for the day
                        new_state = 'COMPLETADA' if up_status.loc[up_id, 'VOLUME_RESTANTE'] <= 1e-6 else 'ACTIVA'
                        up_status.loc[up_id, 'ESTADO'] = new_state

                        # Record the schedule entry
                        schedule_entries.append({
                            'UP': up_id, 'FAZENDA': up_data['FAZENDA'], 'TRANSPORTADOR': transporter,
                            'DIA': current_day, 'MES': current_month, 'DB': up_data['DB'],
                            'RSP': up_data['RSP'], 'QTD_VEICULOS': assigned_vehicles,
                            'VOLUME': transported_vol
                        })
                        accumulated_volume_day += transported_vol

                except (IndexError, KeyError) as e:
                     # print(f"Warn: Data missing for {up_id}/{transporter}, day {current_day}. Skipping entry. Error: {e}")
                     continue
                except Exception as e:
                     print(f"Error processing UP {up_id}, Day {current_day}: {e}")
                     continue

    # --- End of Horizon ---
    final_schedule = pd.DataFrame(schedule_entries)
    return final_schedule

# --- Pattern-Based Heuristic Components ---

def validate_single_day(schedule_df_day, df_factory_day, df_fleet, df_crane):
    """Validates essential constraints for a single day's schedule slice."""
    # Handle empty schedule case
    if schedule_df_day is None or schedule_df_day.empty:
        if df_factory_day is None or df_factory_day.empty or 'DEMANDA_MIN' not in df_factory_day.columns:
            return False # Cannot validate without factory data
        # Empty schedule is feasible only if min demand is zero
        return df_factory_day['DEMANDA_MIN'].iloc[0] <= 1e-6

    # Basic checks on input dataframes
    if 'DIA' not in schedule_df_day.columns: return False
    if not isinstance(df_factory_day, pd.DataFrame) or df_factory_day.empty: return False

    try: # Ensure day value is usable
        day = int(schedule_df_day['DIA'].iloc[0])
    except: return False

    # Filter factory constraints (should already be filtered, but double-check)
    factory_constraints = df_factory_day[df_factory_day['DIA'] == day]
    if factory_constraints.empty: return False

    # Run essential validations (suppress internal prints with flag=0)
    is_feasible = True
    try:
        # Check functions exist (optional, assumes script structure)
        # required_funcs = ['validate_daily_demand', 'validate_daily_rsp', ...]
        # if not all(...): return False

        temp_schedule = schedule_df_day.copy() # Validate on a copy
        if not validate_daily_demand(temp_schedule, factory_constraints, 0): is_feasible = False
        if not validate_daily_rsp(temp_schedule, factory_constraints, 0): is_feasible = False
        if not validate_transporter_fleet_size(temp_schedule, df_fleet): is_feasible = False
        if not validate_crane_capacity(temp_schedule, df_crane): is_feasible = False
        if not validate_fazenda_exclusivity(temp_schedule): is_feasible = False
    except Exception as e:
        print(f"Error during single day validation (Day {day}): {e}")
        return False

    return is_feasible

def find_feasible_daily_pattern(target_day, current_up_status, df_horizon, df_fleet, df_crane, df_factory, df_route, max_attempts=100):
    """Tries to find a feasible schedule pattern for a target day."""
    # Prepare single-day inputs for the constructive heuristic
    horizon_day_info = df_horizon[df_horizon['DIA'] == target_day]
    if horizon_day_info.empty: return None, False
    month_val = horizon_day_info['MES'].iloc[0]
    year_val = horizon_day_info.get('ANO', pd.Series([None])).iloc[0] # Handle missing ANO
    df_horizon_day = pd.DataFrame([{'DIA': target_day, 'MES': month_val, 'ANO': year_val}])
    df_factory_day = df_factory[df_factory['DIA'] == target_day].copy()
    if df_factory_day.empty: return None, False

    last_attempt_schedule = None
    for attempt in range(max_attempts):
        # Use a copy of the current state for the heuristic run
        up_status_for_heuristic = current_up_status.copy().reset_index()
        daily_schedule_attempt = None

        try:
            # Check if heuristic function exists
            if 'constructive_heuristic' not in globals() or not callable(globals()['constructive_heuristic']):
                 print("FATAL: `constructive_heuristic` not defined."); return None, False

            # Run heuristic for the single day
            daily_schedule_attempt = constructive_heuristic(
                df_horizon_day, up_status_for_heuristic, df_fleet, df_crane,
                df_factory_day, df_route, flag=0
            )
            last_attempt_schedule = daily_schedule_attempt # Keep track of last try

        except Exception as e:
            print(f"Error in constructive_heuristic (Day {target_day}, Att {attempt+1}): {e}")
            continue # Try again

        # Filter result for the target day (should be redundant but safe)
        if daily_schedule_attempt is not None and not daily_schedule_attempt.empty:
             if 'DIA' in daily_schedule_attempt.columns:
                 daily_schedule_attempt = daily_schedule_attempt[daily_schedule_attempt['DIA'] == target_day].copy()
             else: daily_schedule_attempt = pd.DataFrame() # Treat as empty if DIA missing

        # Validate this attempt
        if validate_single_day(daily_schedule_attempt, df_factory_day, df_fleet, df_crane):
            return daily_schedule_attempt, True # Return first feasible pattern

    # If no feasible pattern found after max attempts
    print(f"Warn: No feasible pattern found for Day {target_day} after {max_attempts} attempts. Using last.")
    is_last_feasible = validate_single_day(last_attempt_schedule, df_factory_day, df_fleet, df_crane)
    return last_attempt_schedule, is_last_feasible # Return last attempt and its feasibility


def calculate_extension_days(daily_pattern, current_up_status, horizon_end_day, current_day):
    """Calculates how many days a feasible pattern can be repeated based on UP volume."""
    if daily_pattern is None or daily_pattern.empty:
        # Extend empty pattern by 1 day (if within horizon)
        return min(1, max(0, horizon_end_day - current_day + 1))

    if 'UP' not in daily_pattern.columns or 'VOLUME' not in daily_pattern.columns:
        print("Error: Pattern missing UP/VOLUME columns in calculate_extension_days.")
        return 0

    volume_per_up = daily_pattern.groupby('UP')['VOLUME'].sum()
    if volume_per_up.sum() <= 1e-6: # Pattern transports negligible volume
        return min(1, max(0, horizon_end_day - current_day + 1))

    max_k = float('inf')
    for up_id, daily_vol in volume_per_up.items():
        if daily_vol <= 1e-6: continue
        if up_id not in current_up_status.index: return 0 # Required UP missing

        remaining_vol = current_up_status.loc[up_id, 'VOLUME_RESTANTE']
        if remaining_vol <= 1e-6: return 0 # Required UP is empty

        days_possible = math.floor(remaining_vol / daily_vol)
        max_k = min(max_k, days_possible)

    days_left_horizon = max(0, horizon_end_day - current_day + 1)
    final_k = min(max_k, days_left_horizon)

    # Ensure we can apply at least once if some volume remains
    if final_k == 0 and volume_per_up.sum() > 1e-6:
        # Check if *any* UP required by the pattern still has volume
        can_apply_once = any(current_up_status.loc[up, 'VOLUME_RESTANTE'] > 1e-6
                             for up in volume_per_up[volume_per_up > 1e-6].index
                             if up in current_up_status.index)
        if can_apply_once:
            final_k = 1

    return max(0, int(final_k)) # Return non-negative integer


# --- Main Pattern-Based Heuristic Loop ---
def run_pattern_based_heuristic(df_horizon, df_up_database, df_fleet, df_crane, df_factory, df_route, max_daily_attempts=100):
    """Orchestrates the pattern-finding and extension heuristic."""
    print("--- Running Pattern-Based Heuristic ---")
    all_schedule_entries = []

    # Input checks
    if 'DIA' not in df_horizon.columns or df_horizon.empty: return None
    if 'UP' not in df_up_database.columns or 'VOLUME' not in df_up_database.columns: return None
    current_day = df_horizon['DIA'].min()
    horizon_end_day = df_horizon['DIA'].max()

    # Initialize UP status
    up_status = df_up_database.set_index('UP').copy()
    up_status['VOLUME_RESTANTE'] = up_status['VOLUME']
    up_status['DIAS_TRABAJADOS'] = [[] for _ in range(len(up_status))]
    up_status['BLOQUES_TRABAJO'] = 0
    up_status['ESTADO'] = 'PENDIENTE'

    while current_day <= horizon_end_day:
        print(f"\nProcessing Day {current_day}...")

        # 1. Find daily pattern (feasible or best-effort)
        daily_pattern, is_feasible = find_feasible_daily_pattern(
            current_day, up_status, df_horizon, df_fleet, df_crane, df_factory, df_route, max_daily_attempts
        )

        if daily_pattern is None:
             print(f"Error: Heuristic failed completely for Day {current_day}. Skipping.")
             current_day += 1; continue

        # 2. Calculate extension days (k)
        if is_feasible:
             k = calculate_extension_days(daily_pattern, up_status, horizon_end_day, current_day)
             if k == 0:
                 print(f"Warn: Feasible pattern Day {current_day} cannot extend (k=0). Applying once.")
                 k = 1
             else: print(f"Feasible pattern Day {current_day}. Extending {k} day(s).")
        else:
             print(f"Warn: Using infeasible pattern for Day {current_day}. Applying once.")
             k = 1 # Apply infeasible pattern only once

        # Ensure k is valid and within horizon
        k = min(k, horizon_end_day - current_day + 1)
        k = max(0, k)
        if k == 0:
            print(f"Info: k=0 for Day {current_day}. Skipping application.")
            current_day += 1; continue

        # 3. Apply pattern for k days and update state
        for i in range(k):
            day_to_apply = current_day + i
            if day_to_apply > horizon_end_day: break
            if not isinstance(daily_pattern, pd.DataFrame): break # Safety check

            pattern_copy = daily_pattern.copy()
            if not pattern_copy.empty:
                 # Update day/month/year
                 pattern_copy['DIA'] = day_to_apply
                 h_info = df_horizon[df_horizon['DIA'] == day_to_apply]
                 if not h_info.empty:
                      pattern_copy['MES'] = h_info['MES'].iloc[0]
                      if 'ANO' in h_info.columns: pattern_copy['ANO'] = h_info['ANO'].iloc[0]
                 all_schedule_entries.append(pattern_copy)

                 # Update UP state
                 if 'UP' not in pattern_copy.columns or 'VOLUME' not in pattern_copy.columns: continue
                 vol_today = pattern_copy.groupby('UP')['VOLUME'].sum()
                 for up_id, vol in vol_today.items():
                     if up_id in up_status.index and vol > 1e-6:
                         try:
                             up_status.loc[up_id, 'VOLUME_RESTANTE'] -= vol
                             if up_status.loc[up_id, 'VOLUME_RESTANTE'] < 0: up_status.loc[up_id, 'VOLUME_RESTANTE'] = 0

                             # Update work days/blocks
                             days_list = up_status.at[up_id, 'DIAS_TRABAJADOS']
                             if not isinstance(days_list, list): days_list = []
                             last_day = days_list[-1] if days_list else None
                             is_new_block = (last_day is None) or (day_to_apply != last_day + 1)
                             if is_new_block and last_day is not None:
                                 if pd.isna(up_status.at[up_id, 'BLOQUES_TRABAJO']): up_status.at[up_id, 'BLOQUES_TRABAJO'] = 0
                                 up_status.at[up_id, 'BLOQUES_TRABAJO'] += 1
                             if day_to_apply not in days_list:
                                  days_list.append(day_to_apply)
                                  up_status.at[up_id, 'DIAS_TRABAJADOS'] = days_list

                             # Update status
                             new_state = 'COMPLETADA' if up_status.loc[up_id, 'VOLUME_RESTANTE'] <= 1e-6 else 'ACTIVA'
                             up_status.loc[up_id, 'ESTADO'] = new_state
                         except Exception as e: print(f"Error updating state {up_id} day {day_to_apply}: {e}")

        # 4. Advance current day
        current_day += k # Advance by the number of days applied

    print("\n--- Pattern-Based Heuristic Finished ---")
    if not all_schedule_entries: return pd.DataFrame()
    try: final_schedule = pd.concat(all_schedule_entries, ignore_index=True)
    except Exception as e: print(f"Error concatenating schedule: {e}"); return None

    # Final validation check
    print("\nValidating final generated schedule (essential constraints)...")
    try:
        if 'validate_essential_constraints' in globals() and callable(globals()['validate_essential_constraints']):
            is_final_feasible = validate_essential_constraints(final_schedule, df_factory, df_fleet, df_crane, df_up_database, flag=0)
            status = "Feasible" if is_final_feasible else "INFEASIBLE"
            print(f">>> Final Schedule Status: {status} <<<")
        else: print("Warn: Cannot perform final validation.")
    except Exception as e: print(f"Error during final validation: {e}")

    return final_schedule


In [5]:

# --- Run Heuristic ---
initial_solution = run_pattern_based_heuristic(
    df_horizon, df_up_database, df_fleet, df_crane, df_factory, df_route, max_daily_attempts=200
)

if initial_solution is not None:
    print("\n--- Initial Solution Generated ---")
    print(initial_solution.head())
    print(f"Total entries: {len(initial_solution)}")
else:
    print("\n--- Heuristic failed to generate initial solution ---")

# Calculate initial objective
initial_objective = calculate_objective(initial_solution) if initial_solution is not None else float('inf')
print(f"\nInitial Objective (Avg Daily DB Var): {initial_objective:.4f}")

--- Running Pattern-Based Heuristic ---

Processing Day 1...
Error processing UP S6C421, Day 1: cannot access local variable 'accumulated_volume_day' where it is not associated with a value
Error processing UP S5AW10, Day 1: cannot access local variable 'accumulated_volume_day' where it is not associated with a value
Error processing UP S5AW05, Day 1: cannot access local variable 'accumulated_volume_day' where it is not associated with a value
Error processing UP S5AW10, Day 1: cannot access local variable 'accumulated_volume_day' where it is not associated with a value
Error processing UP S5AW05, Day 1: cannot access local variable 'accumulated_volume_day' where it is not associated with a value
Error processing UP S5AK08, Day 1: cannot access local variable 'accumulated_volume_day' where it is not associated with a value
Error processing UP S5AK05, Day 1: cannot access local variable 'accumulated_volume_day' where it is not associated with a value
Error processing UP S6C335, Day 1: c

In [6]:

# --- Neighborhood Search (Simulated Annealing) ---

def move_change_vehicles(current_schedule, df_route, df_fleet, df_up_database):
    """Neighborhood Move: Randomly change vehicle count for one task."""
    if current_schedule.empty: return current_schedule.copy()
    neighbor = current_schedule.copy()
    try: task_idx = random.choice(neighbor.index)
    except IndexError: return neighbor

    task = neighbor.loc[task_idx]
    req_cols = ['TRANSPORTADOR', 'DIA', 'UP', 'QTD_VEICULOS', 'VOLUME']
    if not all(col in task and pd.notna(task[col]) for col in req_cols): return neighbor

    transp, day, up, current_veh = task['TRANSPORTADOR'], task['DIA'], task['UP'], task['QTD_VEICULOS']

    try:
        f_info = df_fleet[df_fleet['TRANSPORTADOR'] == transp].iloc[0]
        f_min, f_max = f_info['FROTA_MIN'], f_info['FROTA_MAX']
    except IndexError: return neighbor # Skip if no fleet info

    # Current total vehicles for transporter on this day
    current_total = neighbor[(neighbor['DIA'] == day) & (neighbor['TRANSPORTADOR'] == transp)]['QTD_VEICULOS'].sum(skipna=True)

    change = random.choice([-1, 1])
    new_veh_task = max(1, current_veh + change) # Ensure at least 1 vehicle

    # New hypothetical total
    new_total = current_total - current_veh + new_veh_task

    # Check transporter fleet limits
    if not (f_min <= new_total <= f_max):
        return neighbor # Move violates fleet limits

    # Apply change
    neighbor.loc[task_idx, 'QTD_VEICULOS'] = new_veh_task

    # Volume is NOT recalculated here for simplicity. Validation handles implications.
    return neighbor

# Add more sophisticated move functions here (e.g., reassign transporter, shift day)

def simulated_annealing(initial_schedule, df_factory, df_fleet, df_crane, df_up_database, df_route,
                        initial_temp=100, cooling_rate=0.99, max_iterations=1000,
                        max_attempts_per_temp=50):
    """Optimizes a schedule using Simulated Annealing."""
    print("\n--- Starting Simulated Annealing ---")

    # Validate starting point
    try:
        if 'validate_essential_constraints' not in globals() or not callable(globals()['validate_essential_constraints']):
             print("Error: SA needs `validate_essential_constraints`."); return initial_schedule
        if not validate_essential_constraints(initial_schedule, df_factory, df_fleet, df_crane, df_up_database, flag=0):
            print("Error: Initial schedule for SA is infeasible."); return initial_schedule
    except Exception as e:
        print(f"Error validating initial schedule for SA: {e}"); return initial_schedule

    # Initialize SA state
    current_sched = initial_schedule.copy()
    current_obj = calculate_objective(current_sched)
    best_sched = current_sched.copy()
    best_obj = current_obj
    temp = initial_temp

    print(f"Initial Objective: {current_obj:.4f}")

    # SA Loop
    for i in range(max_iterations):
        if temp < 1e-3: print("Temperature threshold reached."); break # Stop condition

        accepted_in_iter = False
        for _ in range(max_attempts_per_temp): # Attempts at current temperature
            # 1. Generate Neighbor
            try:
                move_func = random.choice([move_change_vehicles]) # Add more moves later
                neighbor_sched = move_func(current_sched, df_route, df_fleet, df_up_database)
            except Exception as e: continue # Try again if neighbor generation fails

            # 2. Check Feasibility
            try:
                is_feasible = validate_essential_constraints(neighbor_sched, df_factory, df_fleet, df_crane, df_up_database, flag=0)
            except Exception as e: is_feasible = False # Treat validation error as infeasible

            # 3. Evaluate & Accept (if feasible)
            if is_feasible:
                neighbor_obj = calculate_objective(neighbor_sched)
                delta = neighbor_obj - current_obj
                accept = False
                if delta < 0: # Always accept improvement
                    accept = True
                elif temp > 1e-9: # Accept worsening move probabilistically
                    try:
                        prob = math.exp(-delta / temp)
                        if random.random() < prob: accept = True
                    except OverflowError: accept = False

                # 4. Update state if accepted
                if accept:
                    current_sched = neighbor_sched.copy()
                    current_obj = neighbor_obj
                    accepted_in_iter = True
                    if current_obj < best_obj: # Update best found
                        best_sched = current_sched.copy()
                        best_obj = current_obj
                    break # Move to next temperature iteration

        # 5. Cool down
        temp *= cooling_rate

        # Log progress
        if (i + 1) % 100 == 0:
             print(f"  Iter {i+1}/{max_iterations}, Temp: {temp:.3f}, "
                   f"Current: {current_obj:.4f}, Best: {best_obj:.4f}")

    print(f"--- Simulated Annealing Finished ---")
    print(f"Final Best Objective: {best_obj:.4f}")
    return best_sched

In [7]:

# --- Main Execution & Optimization ---

# Check feasibility of the initial solution before SA
is_initial_feasible = False
if initial_solution is not None and not initial_solution.empty:
    try:
        if 'validate_essential_constraints' in globals() and callable(globals()['validate_essential_constraints']):
             is_initial_feasible = validate_essential_constraints(initial_solution, df_factory, df_fleet, df_crane, df_up_database, flag=0)
             status = "FEASIBLE" if is_initial_feasible else "INFEASIBLE"
             print(f"\nInitial solution validation status: {status}. Proceeding accordingly.")
        else: print("\nWarn: Cannot validate initial solution feasibility for SA.")
    except Exception as e: print(f"Error validating initial solution: {e}")

# Run SA only if starting point is feasible
if is_initial_feasible:
    optimized_solution = simulated_annealing(
        initial_schedule=initial_solution,
        df_factory=df_factory, df_fleet=df_fleet, df_crane=df_crane,
        df_up_database=df_up_database, df_route=df_route,
        initial_temp=5,           # Starting temperature
        cooling_rate=0.995,       # Cooling factor
        max_iterations=1000,      # Number of iterations
        max_attempts_per_temp=100 # Attempts per temperature level
    )
else:
    print("\nSkipping SA optimization due to infeasible/missing initial solution.")
    optimized_solution = initial_solution if initial_solution is not None else pd.DataFrame()

# --- Final Output & Save ---
print("\n--- Final Resulting Schedule ---")
if optimized_solution is not None and not optimized_solution.empty:
    # Clean up temporary columns before saving/displaying final
    cols_to_drop = ['RSP_WEIGHTED_SUM', 'ANO'] # Add others if created temporarily
    final_output = optimized_solution.drop(columns=cols_to_drop, errors='ignore')

    print(final_output)
    try:
        final_output.to_excel("optimized_schedule_output.xlsx", index=False)
        print("\nOptimized schedule saved to 'optimized_schedule_output.xlsx'")
        # Also save initial solution for comparison (if it exists)
        if initial_solution is not None and not initial_solution.empty:
             initial_solution.drop(columns=cols_to_drop, errors='ignore').to_excel("initial_schedule_output.xlsx", index=False)
             print("Initial schedule saved to 'initial_schedule_output.xlsx'")

    except Exception as e:
        print(f"\nError saving output files: {e}")

    final_obj = calculate_objective(final_output)
    print(f"\nFinal Objective Value: {final_obj:.4f}")
else:
    print("No final schedule generated.")



Initial solution validation status: FEASIBLE. Proceeding accordingly.

--- Starting Simulated Annealing ---
Initial Objective: 59.8638
  Iter 100/1000, Temp: 3.029, Current: 59.8638, Best: 59.8638
  Iter 200/1000, Temp: 1.835, Current: 59.8638, Best: 59.8638
  Iter 300/1000, Temp: 1.111, Current: 59.8638, Best: 59.8638
  Iter 400/1000, Temp: 0.673, Current: 59.8638, Best: 59.8638
  Iter 500/1000, Temp: 0.408, Current: 59.8638, Best: 59.8638
  Iter 600/1000, Temp: 0.247, Current: 59.8638, Best: 59.8638
  Iter 700/1000, Temp: 0.150, Current: 59.8638, Best: 59.8638
  Iter 800/1000, Temp: 0.091, Current: 59.8638, Best: 59.8638
  Iter 900/1000, Temp: 0.055, Current: 59.8638, Best: 59.8638
  Iter 1000/1000, Temp: 0.033, Current: 59.8638, Best: 59.8638
--- Simulated Annealing Finished ---
Final Best Objective: 59.8638

--- Final Resulting Schedule ---
         UP                      FAZENDA TRANSPORTADOR  DIA  MES          DB  \
0    S6C421                      SIRIEMA         Tover    1   